# Databricks Unity Catalog (UC)

This notebook shows how to use UC functions as LangChain tools, with both LangChain and LangGraph agent APIs.

See Databricks documentation ([AWS](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-create-sql-function.html)|[Azure](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/sql-ref-syntax-ddl-create-sql-function)|[GCP](https://docs.gcp.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-create-sql-function.html)) to learn how to create SQL or Python functions in UC. Do not skip function and parameter comments, which are critical for LLMs to call functions properly.

In this example notebook, we create a simple Python function that executes arbitrary code and use it as a LangChain tool:

```sql
CREATE FUNCTION main.tools.python_exec (
  code STRING COMMENT 'Python code to execute. Remember to print the final result to stdout.'
)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Executes Python code and returns its stdout.'
AS $$
  import sys
  from io import StringIO
  stdout = StringIO()
  sys.stdout = stdout
  exec(code)
  return stdout.getvalue()
$$
```

It runs in a secure and isolated environment within a Databricks SQL warehouse.

In [0]:
%pip install --upgrade --quiet databricks-sdk langchain-community langgraph mlflow

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
from langchain_community.chat_models.databricks import ChatDatabricks

llm = ChatDatabricks(endpoint="databricks-meta-llama-3-70b-instruct")

In [0]:
from langchain_community.tools.databricks import UCFunctionToolkit
from databricks.sdk import WorkspaceClient

tools = (
    UCFunctionToolkit(
        # You can find the SQL warehouse ID in its UI after creation.
        warehouse_id="dd43ee29fedd958d"
    )
    .include(
        # Include functions as tools using their qualified names.
        # You can use "{catalog_name}.{schema_name}.*" to get all functions in a schema.
        "main.tools.python_exec",
    )
    .get_tools()
)

## LangGraph agent example

In [0]:
from langgraph.prebuilt import create_react_agent
agent = create_react_agent(llm, tools, state_modifier="You are a helpful assistant. Make sure to use tool for information.")
agent.invoke({"messages": [{"role": "user", "content": "36939 * 8922.4"}]})

{'messages': [HumanMessage(content='36939 * 8922.4', additional_kwargs={}, response_metadata={}, id='28bcde49-b0e5-43c1-b86f-3702dd70f285'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_afa03335-acd1-45ba-8eea-d48fd55596ea', 'type': 'function', 'function': {'name': 'main__tools__python_exec', 'arguments': '{"code": "print(36939 * 8922.4)"}'}}]}, response_metadata={'prompt_tokens': 793, 'completion_tokens': 29, 'total_tokens': 822}, id='run-0438f421-3bff-4c0e-b22e-9093ffb68d9e-0', tool_calls=[{'name': 'main__tools__python_exec', 'args': {'code': 'print(36939 * 8922.4)'}, 'id': 'call_afa03335-acd1-45ba-8eea-d48fd55596ea', 'type': 'tool_call'}]),
  ToolMessage(content='{"format": "SCALAR", "value": "329584533.59999996\\n", "truncated": false}', name='main__tools__python_exec', id='6242cbb9-1eee-4a9f-b574-714f920d8636', tool_call_id='call_afa03335-acd1-45ba-8eea-d48fd55596ea'),
  AIMessage(content='The result of the multiplication is:\n\n329,584,533.6', additional

## LangChain agent example

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tool_calling_agent = create_tool_calling_agent(llm, tools, prompt)

In [0]:
agent_executor = AgentExecutor(agent=tool_calling_agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "36939 * 8922.4"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `main__tools__python_exec` with `{'code': 'print(36939 * 8922.4)'}`


{"format": "SCALAR", "value": "329584533.59999996\n", "truncated": false}The result of the multiplication is:

329,584,533.6

> Finished chain.


{'input': '36939 * 8922.4',
 'output': 'The result of the multiplication is:\n\n329,584,533.6'}